In [1]:
import os 
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [6]:
import pytz, datetime
from flourish_caregiver.models import CaregiverOffSchedule
from flourish_child.helper_classes.child_fu_onschedule_helper import ChildFollowUpEnrolmentHelper
from flourish_child.models import Appointment as ChildAppointment
from edc_appointment.models import Appointment as CaregiverAppointment
from edc_base.utils import get_utcnow
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from django.db import IntegrityError
from django.db.models.deletion import ProtectedError

In [5]:
subject_identifier = 'B142-040990639-2'
schedule_name = 'c_fu1_schedule1'
onschedule_model_cls = 'flourish_caregiver.onschedulecohortcfu'

_, new_schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    name=schedule_name,
    onschedule_model=onschedule_model_cls)

onsch = OnScheduleChildCohortCFU.objects.get(subject_identifier='B142-040990639-2-10')
tz = pytz.timezone('Africa/Gaborone')
dt = onsch.onschedule_datetime.astimezone(tz)

new_schedule.put_on_schedule(
            subject_identifier=subject_identifier,
            schedule_name=schedule_name,
            onschedule_datetime=dt,
            base_appt_datetime=dt)

quart = OnScheduleCohortCQuarterly.objects.get(subject_identifier=subject_identifier,)
caregiver_appointment = CaregiverAppointment.objects.filter(
    subject_identifier=subject_identifier,
    schedule_name=quart.schedule_name).exclude(appt_status='new').latest('appt_datetime')

try:
    CaregiverOffSchedule.objects.create(
                        subject_identifier=caregiver_appointment.subject_identifier,
                        schedule_name=caregiver_appointment.schedule_name,
                        offschedule_datetime=dt)
except (CaregiverOffSchedule.DoesNotExist, IntegrityError):
    pass

In [13]:
""" Remove cohort A follow up schedules, not on follow yet.
"""
onsch_a = OnScheduleCohortAFU.objects.get(subject_identifier=subject_identifier)
onsch_quart = OnScheduleCohortAQuarterly.objects.get(subject_identifier=subject_identifier)

ssh_a = SubjectScheduleHistory.objects.get(
    subject_identifier=subject_identifier, schedule_name=onsch_a.schedule_name)
fu_appt = CaregiverAppointment.objects.get(
    subject_identifier=subject_identifier, schedule_name=onsch_a.schedule_name)
try:
    fu_appt.delete()
except ProtectedError:
    pass
else:
    onsch_a.delete()
    ssh_a.delete()

    # Remove offschedule
    offschedule = CaregiverOffSchedule.objects.filter(
            subject_identifier=subject_identifier,
            schedule_name=onsch_quart.schedule_name,)

    offschedule._raw_delete(using=offschedule.db)
    ssh = SubjectScheduleHistory.objects.get(subject_identifier=subject_identifier,
                                             schedule_name=onsch_quart.schedule_name,)
    ssh.offschedule_datetime = None
    ssh.schedule_status = 'onschedule'
    ssh.save()